### COUNTEN Clustering

1 - Enter path to image

2 - Enter path to file containing scripts

3 - Enter pixel density of the image (pixels/micrometer)

4 - Enter sigma to adjust how much smoothing of the image is needed (more background noise requires higher sigma value)

5 - Enter min_samples, the minimum number of neurons in ganglion

In [ ]:
neuron_image_path = "/home/klab/Pictures/inputimages/hucdEN/1128.tif"
output_path = "/home/klab/Desktop/"
script_path = '/home/klab/Desktop/ENCOUNTER/scripts'
pixel_density = 3.2
sigma = 3
min_samples = 3

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import warnings
warnings.filterwarnings('ignore')

import os
os.chdir(script_path)

from scripts import *

import javabridge
import bioformats
javabridge.start_vm(class_path=bioformats.JARS)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 10,10
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

neurons, directory, meta = io.load_TIFF(neuron_image_path, output_path)

javabridge.kill_vm()

### Run to display image

In [ ]:
plt.imshow(neurons)
plt.axis('off')

### Run to process image

In [ ]:
local_maxi, labels, gauss = processing.wide_clusters(neurons, 
                                                sigma=sigma, 
                                                pixel_density=pixel_density,
                                                min_samples=min_samples)

del neurons 
ganglion_prop = processing.segmentation(gauss, local_maxi, labels, meta, directory, save = True)
del gauss

### Run to display dataframe 

In [ ]:
df, dist = analysis.create_dataframe(ganglion_prop, labels, local_maxi, meta, directory, save=True)
df